In [8]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import figure as fig

import random


from sklearn.utils import shuffle

from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical

from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img

from skimage.util import img_as_float
from skimage.util import img_as_ubyte

# Models
from keras.models import Sequential
from keras.models import Model

# Layers
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout

# Activation functions
from keras.layers import ReLU
from keras.layers import LeakyReLU
from keras.layers import PReLU
from keras.layers import Softmax

# Callbacks
from keras.callbacks import EarlyStopping

# Optimizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam

# HP tuners
from tensorflow.keras.optimizers.schedules import ExponentialDecay
# Performancec evaluation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [9]:
# Creating the dset
classes = {0: 'Zero', 1: 'One', 2: 'Two', 3: 'Three', 4: 'Four', 5: 'Five', 
          6: 'Six', 7: 'Seven', 8: 'Eight', 9: 'Nine'}

def getClassDistribution(mode, classes=classes):
    """Return a pandas dataframe with class distribution information."""
    class_frequencies  = [list(mode).count(label) for label in classes.keys()]
    class_distribution = [count/len(list(mode)) for count in class_frequencies]
    data = {
        'value_count' : class_frequencies,
        'distribution': class_distribution
    }
    table = pd.DataFrame(data)
    return table

In [10]:
df = pd.read_csv('train.csv').to_numpy()
m = df.shape[0]

# Init inputs and labels
labels = np.array([df[i][0] for i in range(m)]).reshape(-1, 1)
images = []
for index in range(m):
    start = 1
    stop  = 29 
    image = np.zeros((28, 28))
    for row in range(28):
        image[row] = df[index, start:stop]
        start = stop
        stop  = start + 28
    images.append(image)
images = np.array(images)
images = images.reshape(-1, 28, 28, 1)
labels = to_categorical(labels)
print(images.shape, labels.shape)

(42000, 28, 28, 1) (42000, 10)


In [19]:
# Visualize random images from the dset
input_shape = np.shape(images[random.randint(0,42000)])

In [20]:
def getClassDistribution(mode):
    """Return a pandas dataframe with class distribution information."""
    mode = list(np.argmax(mode, axis=1))
    class_frequencies  = [mode.count(label) for label in classes.keys()]
    class_distribution = [count/len(mode) for count in class_frequencies]
    data = {
        'classes'     : classes.values(),
        'value_count' : class_frequencies,
        'distribution': class_distribution
    }
    table = pd.DataFrame(data)
    return table

In [21]:
x_Train, x_Test, y_Train, y_Test = train_test_split(
    images, labels, test_size=0.10)

x_Train, x_Val, y_Train, y_Val = train_test_split(
    x_Train, y_Train, test_size=0.11)

In [22]:
print(x_Train.shape, y_Train.shape)
print(x_Test.shape, y_Test.shape)
print(x_Val.shape, y_Val.shape)

(33642, 28, 28, 1) (33642, 10)
(4200, 28, 28, 1) (4200, 10)
(4158, 28, 28, 1) (4158, 10)


In [14]:
getClassDistribution(y_Train)

,classes,value_count,distribution
0,Zero,3285,0.097646
1,One,3791,0.112687
2,Two,3299,0.098062
3,Three,3512,0.104393
4,Four,3223,0.095803
5,Five,3020,0.089769
6,Six,3332,0.099043
7,Seven,3510,0.104334
8,Eight,3264,0.097022
9,Nine,3406,0.101242


In [15]:
getClassDistribution(y_Test)

,classes,value_count,distribution
0,Zero,433,0.103095
1,One,453,0.107857
2,Two,448,0.106667
3,Three,434,0.103333
4,Four,414,0.098571
5,Five,392,0.093333
6,Six,395,0.094048
7,Seven,450,0.107143
8,Eight,400,0.095238
9,Nine,381,0.090714


In [29]:
mymodel = Sequential()
mymodel.add(Conv2D(input_shape=(28,28,1), filters=2,
                   kernel_size=3, padding='same', activation='relu'))
mymodel.add(Conv2D(filters=4, kernel_size=3, padding='same', activation='relu'))
mymodel.add(Conv2D(filters=8, kernel_size=3, padding='same', activation='relu'))
mymodel.add(Conv2D(filters=12, kernel_size=3, padding='same', activation='relu'))
mymodel.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
mymodel.add(GlobalMaxPooling2D())
mymodel.add(Flatten())
mymodel.add(Dense(units=10, activation=Softmax()))
mymodel.compile(optimizer=Adam(), loss='categorical_crossentropy',
                metrics='accuracy')
mymodel.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 28, 28, 2)         20        
                                                                 
 conv2d_15 (Conv2D)          (None, 28, 28, 4)         76        
                                                                 
 conv2d_16 (Conv2D)          (None, 28, 28, 8)         296       
                                                                 
 conv2d_17 (Conv2D)          (None, 28, 28, 12)        876       
                                                                 
 conv2d_18 (Conv2D)          (None, 28, 28, 16)        1744      
                                                                 
 global_max_pooling2d_4 (Glo  (None, 16)               0         
 balMaxPooling2D)                                                
                                                      

In [30]:
mymodel.fit(x_Train, y_Train, validation_data=(x_Val, y_Val), epochs=10)

Epoch 1/10
1052/1052 [==============================] - 11s 10ms/step - loss: 1.6952 - accuracy: 0.5694 - val_loss: 0.7699 - val_accuracy: 0.7607
Epoch 2/10
1052/1052 [==============================] - 11s 10ms/step - loss: 0.5886 - accuracy: 0.8184 - val_loss: 0.4518 - val_accuracy: 0.8600
Epoch 3/10
1052/1052 [==============================] - 11s 10ms/step - loss: 0.3979 - accuracy: 0.8770 - val_loss: 0.4196 - val_accuracy: 0.8634
Epoch 4/10
1052/1052 [==============================] - 11s 10ms/step - loss: 0.3040 - accuracy: 0.9059 - val_loss: 0.2474 - val_accuracy: 0.9209
Epoch 5/10
1052/1052 [==============================] - 11s 10ms/step - loss: 0.2508 - accuracy: 0.9238 - val_loss: 0.2290 - val_accuracy: 0.9271
Epoch 6/10
1052/1052 [==============================] - 11s 10ms/step - loss: 0.2133 - accuracy: 0.9346 - val_loss: 0.2513 - val_accuracy: 0.9214
Epoch 7/10
1052/1052 [==============================] - 11s 10ms/step - loss: 0.1959 - accuracy: 0.9396 - val_loss: 0.1955 -

In [35]:
prediction = mymodel.predict(x_Test)
print(prediction.shape)

(4200, 10)


In [36]:
prediction = np.argmax(prediction, axis=1)

In [37]:
print(prediction.shape)

(4200,)


In [38]:
actual = np.argmax(y_Test, axis=1)
classificationReport = classification_report(actual, prediction, zero_division=0)

In [39]:
print(classificationReport)

              precision    recall  f1-score   support

           0       0.92      0.98      0.95       387
           1       0.99      0.98      0.98       493
           2       0.90      0.94      0.92       410
           3       0.96      0.96      0.96       425
           4       0.97      0.97      0.97       422
           5       0.95      0.94      0.94       363
           6       0.98      0.95      0.96       450
           7       0.96      0.97      0.96       418
           8       0.96      0.90      0.93       395
           9       0.94      0.94      0.94       437

    accuracy                           0.95      4200
   macro avg       0.95      0.95      0.95      4200
weighted avg       0.95      0.95      0.95      4200

